# MyCode Function

In [ ]:
import json
import random
import constraint
from techniques.PaDMTP import padmtp_algo
from techniques.RandomTesting import random_testing
from testcodes.originalCode import code as MyCode
from techniques.AdaptiveRandomTesting import adaptive_random_testing

### PaDMTP

In [ ]:
problem = constraint.Problem()

problem.addVariable('x', range(-6, 7))
problem.addVariable('y', range(-6, 7))

problem.addConstraint(MyCode, ['x', 'y'])

solutions = problem.getSolutions()

length = len(solutions)

In [ ]:
# Cleaning Dataset where values of 'x' or 'y' are 0(zero)

cleaned_solutions = []

for index, solution in enumerate(solutions):
    if solution['x'] == 0 or solution['y'] == 0:
        continue
    else:
        cleaned_solutions.append(solutions[index])
        
original_length = len(cleaned_solutions)

print("\nCleaned Possible Solutions Dataset: \n(x,y) ∈ {", end="")
for index, solution in enumerate(cleaned_solutions):
    if index == original_length - 1:
        print("({},{})".format(solution['x'], solution['y']), end="")
    else:
        print("({},{}),".format(solution['x'], solution['y']), end="")
print("}\n")

In [ ]:
print(f"MyCode - PaDMTP - Test Cases: {original_length}")

In [ ]:
mutPyObj = 'mut.py --target testcodes.originalCode --unit-test mutations.test_original_padmt -m'

padmtp_rslt = padmtp_algo(MyCode, cleaned_solutions, mutPyObj)

In [ ]:
padmtp_rslt['PaDMTP_Overhead']

In [ ]:
padmtp_rslt['PaDMTP_DataFrame']

In [ ]:
PaDMTP_MT_Scores = []

for index, row in padmtp_rslt['PaDMTP_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    PaDMTP_MT_Scores.append(val)

### Random Testing (RT)

In [ ]:
# Generating random inputs
rt_test_cases = []
for i in range(original_length):
    x = random.randint(-100, 101)
    y = random.randint(-100, 101)
    
    rt_test_cases.append([x, y])

In [ ]:
print(f"MyCode - Random Testing - Test Cases: {len(rt_test_cases)}")

In [ ]:
# Saving RT Test Cases to JSON File

jsonString = json.dumps(rt_test_cases)
jsonFile = open("./temp/RT_Test_Cases.json", 'w')
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
pytest_cmd = 'pytest techniques/rt_original.py'
mutpy_cmd = 'mut.py --target testcodes.originalCode --unit-test mutations.test_original_rt -m'

rt_rslt = random_testing(pytest_cmd, mutpy_cmd, rt_test_cases)

In [ ]:
rt_rslt['RT_Overhead']

In [ ]:
rt_rslt['RT_DataFrame']

In [ ]:
RT_MT_Scores = []

for index, row in rt_rslt['RT_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    RT_MT_Scores.append(val)

### Adaptive Random Testing (ART)

In [ ]:
# Generating random inputs
art_test_cases = []
for i in range(original_length):
    x = random.randint(-100, 101)
    y = random.randint(-100, 101)
    
    art_test_cases.append([x, y])

In [ ]:
print(f"MyCode - Adaptive Random Testing - Test Cases: {len(art_test_cases)}")

In [ ]:
mutpy_cmd = 'mut.py --target testcodes.originalCode --unit-test mutations.test_original_art -m'

art_rslt = adaptive_random_testing(MyCode, mutpy_cmd, art_test_cases, 'MyCode')

In [ ]:
art_rslt['ART_Overhead']

In [ ]:
art_rslt['ART_DataFrame']

In [ ]:
ART_MT_Scores = []

for index, row in art_rslt['ART_DataFrame'].iterrows():
    val = row['MT Score']
    val = val.replace("%", "")
    val = float(val) / 100
    val = round(val, 3)
    
    ART_MT_Scores.append(val)

### Pairwise T-Test

In [ ]:
import numpy as np
from scipy.stats import ttest_rel
from tabulate import tabulate

PaDMTP = np.array(PaDMTP_MT_Scores)
RT = np.array(RT_MT_Scores)
ART = np.array(ART_MT_Scores)

# Perform pairwise t-tests
t_stat_PaDMTP_RT, p_value_PaDMTP_RT = ttest_rel(PaDMTP, RT)
t_stat_PaDMTP_ART, p_value_PaDMTP_ART = ttest_rel(PaDMTP, ART)

# Create Table 1
table1_data = [
    ["Test", "Mean", "Variance", "Observations", "df", "t Stat"],
    ["PaDMTP vs RT", np.mean(PaDMTP), np.var(PaDMTP), len(PaDMTP), len(PaDMTP) - 1, t_stat_PaDMTP_RT],
    ["PaDMTP vs ART", np.mean(PaDMTP), np.var(PaDMTP), len(PaDMTP), len(PaDMTP) - 1, t_stat_PaDMTP_ART]
]
table1 = tabulate(table1_data, headers="firstrow", tablefmt="grid")

# Create Table 2
table2_data = [
    ["Test", "P(T<=t) one-tail", "t Critical one-tail", "P(T<=t) two-tail", "t Critical two-tail"],
    ["PaDMTP vs RT", p_value_PaDMTP_RT/2, -abs(t_stat_PaDMTP_RT), p_value_PaDMTP_RT, -abs(t_stat_PaDMTP_RT)],
    ["PaDMTP vs ART", p_value_PaDMTP_ART/2, -abs(t_stat_PaDMTP_ART), p_value_PaDMTP_ART, -abs(t_stat_PaDMTP_ART)]
]
table2 = tabulate(table2_data, headers="firstrow", tablefmt="grid")

# Print the tables
print("Table 1:")
print(table1)
print()
print("Table 2:")
print(table2)

# Perform pairwise t-tests
t_stat_PaDMTP_RT, p_value_PaDMTP_RT = ttest_rel(PaDMTP, RT)
t_stat_PaDMTP_ART, p_value_PaDMTP_ART = ttest_rel(PaDMTP, ART)

# Print the results
print("\n\nPaDMTP vs RT:")
print("t-statistic:", t_stat_PaDMTP_RT)
print("p-value:", p_value_PaDMTP_RT)

print("\nPaDMTP vs ART:")
print("t-statistic:", t_stat_PaDMTP_ART)
print("p-value:", p_value_PaDMTP_ART)